In [4]:
import json
import os
import sys
import pickle
import re

import numpy as np
import pandas as pd
from networks import DualNet, DualNetEndToEnd, PrimalNet, PrimalNetEndToEnd
import torch

def evaluate(data, primal_net):        
    X = data.X[data.test_indices]
    Y_target = data.Y[data.test_indices]
    # Forward pass through networks
    Y = primal_net(X)

    ineq_dist = data.ineq_dist(X, Y)
    eq_resid = data.eq_resid(X, Y)

    # Convert lists to arrays for easier handling
    obj_values = data.obj_fn(X, Y).detach().numpy()
    ineq_max_vals = torch.max(ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    ineq_mean_vals = torch.mean(ineq_dist, dim=1).detach().numpy()
    eq_max_vals = torch.max(torch.abs(eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    eq_mean_vals = torch.mean(torch.abs(eq_resid), dim=1).detach().numpy()
    known_obj = data.obj_fn(X, Y_target).detach().numpy()
    # obj_values is negative
    opt_gap = (obj_values - known_obj)/np.abs(known_obj) * 100

    return np.mean(obj_values), np.mean(known_obj), np.mean(opt_gap), np.mean(ineq_max_vals), np.mean(ineq_mean_vals), np.mean(eq_max_vals), np.mean(eq_mean_vals)

def dual_evaluate(data, dual_net):
    X = data.X[data.test_indices]
    target_mu = data.mu[data.test_indices]
    target_lamb = data.lamb[data.test_indices]
    # Forward pass through networks
    mu, lamb = dual_net(X)

    obj_values = data.dual_obj_fn(X, mu, lamb).detach().numpy()
    known_obj = data.dual_obj_fn(X, target_mu, target_lamb).detach().numpy()
    dual_ineq_dist = data.dual_ineq_dist(mu, lamb)
    dual_eq_resid = data.dual_eq_resid(mu, lamb)

    opt_gap = (obj_values - known_obj)/np.abs(known_obj) * 100

    ineq_max_vals = torch.max(dual_ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    # eq_max_vals = torch.max(torch.abs(dual_eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    ineq_mean_vals = torch.mean(dual_ineq_dist, dim=1).detach().numpy()
    # eq_mean_vals = torch.mean(torch.abs(dual_eq_resid), dim=1).detach().numpy()

    return np.mean(obj_values), np.mean(known_obj), np.mean(opt_gap), np.mean(ineq_max_vals), np.mean(ineq_mean_vals), np.mean([0.0]), np.mean([0.0])

repeats = 5
stats_dict = {}

for experiment in ["simple", "row", "column", "random", "obj"]:
    stats_dict[experiment] = {"predicted_obj": [], "known_obj": [], "opt_gap": [], "ineq_max": [], "ineq_mean": [], "eq_max": [], "eq_mean": []}
    random_params = ["tau_0.9_rho_10.0_rhomax_50000_alpha_1.5", "tau_0.9_rho_0.1_rhomax_5000_alpha_1.0", "tau_0.7_rho_0.1_rhomax_50000_alpha_2.0", "tau_0.5_rho_1.0_rhomax_10000_alpha_10.0", "tau_0.9_rho_10.0_rhomax_50000_alpha_10.0"]
    for i, repeat in enumerate(range(repeats)):
        
        directory = f"experiment-output/ch4/ch4-reproduction-random-params/{experiment}/{random_params[i]}_repeat:{repeat}"
        # directory = f"experiment-output/ch5-reproduction-nonconvex/{experiment}/repeat:{repeat}"
        data_path = f"data/QP_data/QP_type:{experiment}_var:100_ineq:50_eq:50_num_samples:10000.pkl"
        print(data_path)
        # List files and extract numeric prefixes
        files = os.listdir(directory)
        pattern = re.compile(r"^([\d.]+)_best_primal_net\.pth$")

        # Filter and convert to float
        numbered_files = [(float(m.group(1)), f) for f in files if (m := pattern.match(f))]

        # Get file with smallest number
        min_file = min(numbered_files, key=lambda x: x[0])[1]

        args = json.load(open('config.json'))
        data = pickle.load(open(data_path, 'rb'))
        primal_net = PrimalNet(args, data=data)
        primal_net.load_state_dict(torch.load(os.path.join(directory, min_file), weights_only=True))

        obj_val, known_obj, opt_gap, ineq_max, ineq_mean, eq_max, eq_mean = evaluate(data, primal_net)
        stats_dict[experiment]["predicted_obj"].append(obj_val)
        stats_dict[experiment]["known_obj"].append(known_obj)
        stats_dict[experiment]["opt_gap"].append(opt_gap)
        stats_dict[experiment]["ineq_max"].append(ineq_max)
        stats_dict[experiment]["ineq_mean"].append(ineq_mean)
        stats_dict[experiment]["eq_max"].append(eq_max)
        stats_dict[experiment]["eq_mean"].append(eq_mean)


stats_dict

        

data/QP_data/QP_type:simple_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:simple_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:simple_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:simple_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:simple_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:row_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:row_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:row_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:row_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:row_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:column_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:column_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:column_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP_data/QP_type:column_var:100_ineq:50_eq:50_num_samples:10000.pkl
data/QP

{'simple': {'predicted_obj': [-14.919759138518534,
   -15.041652945077244,
   -14.964690095515898,
   -14.975709280914536,
   -14.81955487375971],
  'known_obj': [-15.037454693695281,
   -15.037454693695281,
   -15.037454693695281,
   -15.037454693695281,
   -15.037454693695281],
  'opt_gap': [0.7834218060544529,
   -0.02859135175651931,
   0.48513927982729277,
   0.41104900085924667,
   1.4500587540260403],
  'ineq_max': [0.010907337533850765,
   0.04222041990634877,
   0.03255320309219123,
   0.0010520906696660055,
   0.008160390830264408],
  'ineq_mean': [0.0006831285436810389,
   0.003626927635663962,
   0.001268154386377111,
   3.705330681345522e-05,
   0.0002051003682463282],
  'eq_max': [0.010984893116720263,
   0.04380366856908005,
   0.022784673071669762,
   0.007089928758906608,
   0.004828305966934044],
  'eq_mean': [0.0034382708048531444,
   0.014072168316595473,
   0.007165308147830592,
   0.002215102025881409,
   0.001481273069295882]},
 'row': {'predicted_obj': [-14.6819

In [5]:
# Prepare final summary for LaTeX export
summary = {
    "Experiment": [],
    "Optimal Obj": [],
    "Predicted Obj": [],
    "OptGap (\%)": [],
    "IneqMax": [],
    "IneqMean": [],
    "EqMax": [],
    "EqMean": [],
}

for experiment, metrics in stats_dict.items():
    summary["Experiment"].append(experiment)
    summary["Optimal Obj"].append(f"{np.mean(metrics['known_obj']):.3f}")
    summary["Predicted Obj"].append(f"{np.mean(metrics['predicted_obj']):.3f}({np.std(metrics['predicted_obj']):.3f})")
    summary["OptGap (\%)"].append(
        f"{np.mean(metrics['opt_gap']):.3f}({np.std(metrics['opt_gap']):.3f})"
    )
    summary["IneqMax"].append(
        f"{np.mean(metrics['ineq_max']):.3f}({np.std(metrics['ineq_max']):.3f})"
    )
    summary["IneqMean"].append(
        f"{np.mean(metrics['ineq_mean']):.3f}({np.std(metrics['ineq_mean']):.3f})"
    )
    summary["EqMax"].append(
        f"{np.mean(metrics['eq_max']):.3f}({np.std(metrics['eq_max']):.3f})"
    )
    summary["EqMean"].append(
        f"{np.mean(metrics['eq_mean']):.3f}({np.std(metrics['eq_mean']):.3f})"
    )

df = pd.DataFrame(summary)

# For LaTeX export
latex_table = df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{llllllll}
\toprule
Experiment & Optimal Obj & Predicted Obj & OptGap (\%) & IneqMax & IneqMean & EqMax & EqMean \\
\midrule
simple & -15.037 & -14.944(0.074) & 0.620(0.490) & 0.019(0.016) & 0.001(0.001) & 0.018(0.014) & 0.006(0.005) \\
row & -15.570 & -15.151(0.409) & 2.692(2.627) & 0.094(0.033) & 0.005(0.002) & 0.098(0.016) & 0.031(0.005) \\
column & -15.358 & -14.777(0.417) & 3.752(2.698) & 0.125(0.085) & 0.006(0.005) & 0.069(0.038) & 0.022(0.012) \\
random & -15.602 & -14.764(0.656) & 5.361(4.195) & 0.073(0.046) & 0.003(0.003) & 0.058(0.045) & 0.019(0.014) \\
obj & -15.630 & -14.434(0.163) & 7.687(1.033) & 0.036(0.038) & 0.002(0.002) & 0.013(0.005) & 0.004(0.002) \\
\bottomrule
\end{tabular}



In [ ]:
# Get the quality of dual solutions:
dual_stats_dict = {}
for experiment in ["simple"]:
    dual_stats_dict[experiment] = {"obj": [], "known_obj": [], "opt_gap": [], "ineq_max": [], "ineq_mean": [], "eq_max": [], "eq_mean": []}
    for repeat in range(repeats):
        directory = f"experiment-output/ch4/ch4-reproduction/{experiment}/repeat:{repeat}"
        data_path = f"data/QP_data/QP_type:{experiment}_var:100_ineq:50_eq:50_num_samples:10000.pkl"

        args = json.load(open('config.json'))
        data = pickle.load(open(data_path, 'rb'))
        dual_net = DualNet(args, data=data)
        dual_net.load_state_dict(torch.load(os.path.join(directory, 'dual_weights.pth'), weights_only=True))

        obj_val, known_obj, opt_gap, ineq_max, ineq_mean, eq_max, eq_mean = dual_evaluate(data, dual_net)
        dual_stats_dict[experiment]["obj"].append(obj_val)
        dual_stats_dict[experiment]["known_obj"].append(known_obj)
        dual_stats_dict[experiment]["opt_gap"].append(opt_gap)
        dual_stats_dict[experiment]["ineq_max"].append(ineq_max)
        dual_stats_dict[experiment]["ineq_mean"].append(ineq_mean)
        dual_stats_dict[experiment]["eq_max"].append(eq_max)
        dual_stats_dict[experiment]["eq_mean"].append(eq_mean)

dual_stats_dict

FileNotFoundError: [Errno 2] No such file or directory: 'experiment-output/ch5-reproduction/simple/repeat:0/dual_weights.pth'

In [44]:
# Prepare final summary for LaTeX export
summary = {
    "Optimal Obj": [],
    "Predicted Obj": [],
    "OptGap (\%)": [],
    "IneqMax": [],
    "IneqMean": [],
    "EqMax": [],
    "EqMean": [],
}

for experiment, metrics in dual_stats_dict.items():
    summary["Optimal Obj"].append(f"{np.mean(metrics['known_obj']):.3f}")
    summary["Predicted Obj"].append(f"{np.mean(metrics['obj']):.3e}({np.std(metrics['obj']):.3e})")
    summary["OptGap (\%)"].append(f"{np.mean(metrics['opt_gap']):.3e}({np.std(metrics['opt_gap']):.3e})")
    summary["IneqMax"].append(
        f"{np.mean(metrics['ineq_max']):.3f}({np.std(metrics['ineq_max']):.3f})"
    )
    summary["IneqMean"].append(
        f"{np.mean(metrics['ineq_mean']):.3f}({np.std(metrics['ineq_mean']):.3f})"
    )
    summary["EqMax"].append(
        f"{np.mean(metrics['eq_max']):.3f}({np.std(metrics['eq_max']):.3f})"
    )
    summary["EqMean"].append(
        f"{np.mean(metrics['eq_mean']):.3f}({np.std(metrics['eq_mean']):.3f})"
    )

df = pd.DataFrame(summary)

# For LaTeX export
latex_table = df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{lllllll}
\toprule
Optimal Obj & Predicted Obj & OptGap (\%) & IneqMax & IneqMean & EqMax & EqMean \\
\midrule
-15.037 & -7.778e+03(6.240e+03) & -5.175e+04(4.162e+04) & 0.023(0.006) & 0.003(0.001) & 0.000(0.000) & 0.000(0.000) \\
\bottomrule
\end{tabular}



In [ ]:
import pandas as pd

# Mapping of original metric keys to display names
metric_names = {
    'predicted_obj': 'Predicted',
    'known_obj': 'Optimal',
    'opt_gap': 'Gap (\\%)',
    'ineq_max': 'Ineq. Max',
    'ineq_mean': 'Ineq. Mean',
    'eq_max': 'Eq. Max',
    'eq_mean': 'Eq. Mean'
}

metrics = list(metric_names.keys())
problem_types = list(stats_dict.keys())
repeats = range(len(stats_dict[problem_types[0]][metrics[0]]))

# Prepare multi-indexed rows: (Repeat, Renamed Metric)
rows = []
for r in repeats:
    for m in metrics:
        rows.append((f"Repeat {r}", metric_names[m]))

# Create DataFrame with multi-index rows and problem type columns
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(rows, names=["Repeat", "Metric"]),
                  columns=problem_types)

# Fill in the values
for problem in problem_types:
    for r in repeats:
        for m in metrics:
            df.loc[(f"Repeat {r}", metric_names[m]), problem] = stats_dict[problem][m][r]

# Convert to LaTeX with booktabs
latex_table = df.to_latex(
    escape=False,
    multirow=True,
    column_format='ll' + 'c' * len(problem_types),
    bold_rows=False,
    index_names=True,
    header=True,
    float_format="%.3f",
    caption="Detailed evaluation metrics per repeat and problem type.",
    label="tab:detailed-results",
    longtable=False,
    na_rep=''
)

print(latex_table)


\begin{table}
\caption{Detailed evaluation metrics per repeat and problem type.}
\label{tab:detailed-results}
\begin{tabular}{llccccc}
\toprule
 &  & simple & column & row & random & obj \\
Repeat & Metric &  &  &  &  &  \\
\midrule
\multirow[t]{7}{*}{Repeat 0} & Predicted & -14.920 & -14.454 & -14.682 & -14.343 & -14.462 \\
 & Optimal & -15.037 & -15.358 & -15.570 & -15.602 & -15.630 \\
 & Gap (\%) & 0.783 & 5.842 & 5.702 & 8.041 & 7.494 \\
 & Ineq. Max & 0.011 & 0.072 & 0.062 & 0.057 & 0.017 \\
 & Ineq. Mean & 0.001 & 0.003 & 0.003 & 0.002 & 0.001 \\
 & Eq. Max & 0.011 & 0.105 & 0.104 & 0.112 & 0.015 \\
 & Eq. Mean & 0.003 & 0.033 & 0.033 & 0.036 & 0.005 \\
\cline{1-7}
\multirow[t]{7}{*}{Repeat 1} & Predicted & -15.042 & -15.336 & -15.679 & -15.577 & -14.613 \\
 & Optimal & -15.037 & -15.358 & -15.570 & -15.602 & -15.630 \\
 & Gap (\%) & -0.029 & 0.136 & -0.702 & 0.164 & 6.550 \\
 & Ineq. Max & 0.042 & 0.290 & 0.140 & 0.118 & 0.106 \\
 & Ineq. Mean & 0.004 & 0.016 & 0.008 & 0.006 & 0